In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"

In [3]:
%cd /content/gdrive/My Drive/kaggle
%pwd

/content/gdrive/My Drive/kaggle


'/content/gdrive/My Drive/kaggle'

In [4]:
! kaggle datasets download -d paramaggarwal/fashion-product-images-small

 99% 559M/565M [00:12<00:00, 35.9MB/s]
100% 565M/565M [00:12<00:00, 47.9MB/s]


In [5]:
%cd ~

/root


In [6]:
!unzip "/content/gdrive/My Drive/kaggle/fashion-product-images-small.zip"

Streaming output truncated to the last 5000 lines.
  inflating: myntradataset/images/58131.jpg  
  inflating: myntradataset/images/58132.jpg  
  inflating: myntradataset/images/58133.jpg  
  inflating: myntradataset/images/58135.jpg  
  inflating: myntradataset/images/58136.jpg  
  inflating: myntradataset/images/58137.jpg  
  inflating: myntradataset/images/58138.jpg  
  inflating: myntradataset/images/58139.jpg  
  inflating: myntradataset/images/5814.jpg  
  inflating: myntradataset/images/58140.jpg  
  inflating: myntradataset/images/58141.jpg  
  inflating: myntradataset/images/58143.jpg  
  inflating: myntradataset/images/58144.jpg  
  inflating: myntradataset/images/58145.jpg  
  inflating: myntradataset/images/58146.jpg  
  inflating: myntradataset/images/58147.jpg  
  inflating: myntradataset/images/58148.jpg  
  inflating: myntradataset/images/58149.jpg  
  inflating: myntradataset/images/5815.jpg  
  inflating: myntradataset/images/58150.jpg  
  inflating: myntradataset/imag

In [7]:
%ls

images/  myntradataset/  styles.csv


In [0]:
%rm -r images
%rm styles.csv

In [9]:
%ls

myntradataset/


### Clean and filter metadata

In [0]:
import numpy as np
import pandas as pd
import os

base_path = "myntradataset/"
old_csv_path = os.path.join(base_path, "styles.csv")
csv_path = os.path.join(base_path, "styles_fixed.csv")
img_path = os.path.join(base_path, "images")

In [0]:
# Fixing bad lines in csv file (due to commas in product names)

import csv

with open(old_csv_path) as rf, open(csv_path, 'w') as wf:
    csv_reader = csv.reader(rf, delimiter=',')
    csv_writer = csv.writer(wf, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in csv_reader:
        if len(row) > 10:
            save_row = row[:8]
            save_row.append(','.join(row[9:]))
        else:
            save_row = row
        csv_writer.writerow(save_row)

In [0]:
# %cp myntradataset/styles_fixed.csv "/content/gdrive/My Drive/kaggle/fashion-product-images-small/styles_fixed.csv"

In [0]:
styles = pd.read_csv(csv_path)

In [0]:
# Covert the datatype of column containing articleType to categorical

styles['articleType'] = styles['articleType'].astype('category')

In [33]:
styles['image'] = styles.apply(lambda row: str(row['id']) + ".jpg", axis=1)
print(styles['image'])

0        15970.jpg
1        39386.jpg
2        59263.jpg
3        21379.jpg
4        53759.jpg
           ...    
44441    17036.jpg
44442     6461.jpg
44443    18842.jpg
44444    46694.jpg
44445    51623.jpg
Name: image, Length: 44446, dtype: object


In [34]:
for image in styles['image']:
    if not os.path.exists(os.path.join(img_path, image)):
        print("Image {} doesn't exist!".format(image))

Image 39403.jpg doesn't exist!
Image 39410.jpg doesn't exist!
Image 39401.jpg doesn't exist!
Image 39425.jpg doesn't exist!
Image 12347.jpg doesn't exist!


In [35]:
# Filer out rows for which images don't exist
img_exists = styles.apply(lambda row: os.path.exists(os.path.join(img_path, row['image'])), axis=1)

styles = styles[img_exists]
print(styles)

          id gender  ...                               productDisplayName      image
0      15970    Men  ...                 Turtle Check Men Navy Blue Shirt  15970.jpg
1      39386    Men  ...               Peter England Men Party Blue Jeans  39386.jpg
2      59263  Women  ...                         Titan Women Silver Watch  59263.jpg
3      21379    Men  ...    Manchester United Men Solid Black Track Pants  21379.jpg
4      53759    Men  ...                            Puma Men Grey T-shirt  53759.jpg
...      ...    ...  ...                                              ...        ...
44441  17036    Men  ...                        Gas Men Caddy Casual Shoe  17036.jpg
44442   6461    Men  ...               Lotto Men's Soccer Track Flip Flop   6461.jpg
44443  18842    Men  ...             Puma Men Graphic Stellar Blue Tshirt  18842.jpg
44444  46694  Women  ...                   Rasasi Women Blue Lady Perfume  46694.jpg
44445  51623  Women  ...  Fossil Women Pink Dial Chronograph Watc

### Create train-test splits

In [36]:
# Place images from even-numbered years in train-split and odd-numbered years in test-split

is_train = styles['year']%2==0
is_test = styles['year']%2!=0

full_train = styles[is_train]
print(full_train.shape)

full_test = styles[is_test]
print(full_test.shape)

(23787, 11)
(20654, 11)


### Sub-split training data for pre-training and fine-tuning

In [37]:
# Find the top-20 classes in the training split

top_articleType = full_train.groupby('articleType').size().sort_values(ascending=False).head(20).reset_index()

print("Top 20 classes:")
print(top_articleType)

Top 20 classes:
     articleType     0
0        Tshirts  2749
1        Watches  2482
2   Casual Shoes  1551
3         Kurtas  1140
4         Shirts  1128
5           Tops  1059
6     Sunglasses  1042
7       Handbags  1006
8   Sports Shoes   881
9          Heels   775
10        Briefs   662
11       Wallets   600
12    Flip Flops   508
13         Socks   505
14         Belts   492
15       Sandals   477
16        Sarees   427
17       Dresses   371
18  Formal Shoes   359
19         Flats   342


In [38]:
# Pick the rows corresponding to the top-20 classes for pretraining

filter_topArticles = full_train['articleType'].isin(top_articleType['articleType'])
train_data = full_train[filter_topArticles]
print(train_data.shape)

(18556, 11)


In [39]:
# Pick the rows corresponding to the rest of the classes for fine-tuning

fine_tune_data = full_train[~filter_topArticles]
print(fine_tune_data.shape)

(5231, 11)


In [0]:
test_data = full_test

In [0]:
# Dictionary that maps category number to class name

class_map = dict(zip(styles['articleType'].cat.codes, styles['articleType']))

### Load Dataset

In [0]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from PIL import Image

class FashionDatasetSmall(Dataset):
    """
    Custom dataset class that uses the metadata stored in a DataFrame and returns
    the actual data: (x,y) = (img, label) -
        img is a Tensor representing the image
        label is the categorical label corresponding to the articleType
    """
    def __init__(self, df, img_path):
        """
        Args:
            df (DataFrame): pandas DataFrame containing the metadata
            img_path (string): path to the folder where images are
        """
        self.img_path = img_path
        self.image_arr = np.asarray(df['image'].values)
        self.label_arr = np.asarray(df['articleType'].cat.codes.values)
        self.to_tensor = transforms.ToTensor()

    def __getitem__(self, index):
        img_name = self.image_arr[index]
        img_as_img = Image.open(os.path.join(self.img_path, img_name))
        img_as_tensor = self.to_tensor(img_as_img)

        label = self.label_arr[index]

        return (img_as_tensor, label)

    def __len__(self):
        return len(self.image_arr)

In [64]:
train_dataset = FashionDatasetSmall(train_data, img_path)
ft_dataset = FashionDatasetSmall(fine_tune_data, img_path)
test_dataset = FashionDatasetSmall(test_data, img_path)

print("Loaded training dataset with {} samples".format(len(train_dataset)))
print("Loaded fine-tuning dataset with {} samples".format(len(ft_dataset)))
print("Loaded test dataset with {} samples".format(len(test_dataset)))

Loaded training dataset with 18556 samples
Loaded fine-tuning dataset with 5231 samples
Loaded test dataset with 20654 samples


In [0]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
ft_loader = DataLoader(ft_dataset, batch_size=4, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=4)

In [61]:
for item in train_loader:
    print(item[0].size())
    print(item[1].size())
    break

torch.Size([4, 3, 80, 60])
torch.Size([4])
